# Extraction, Transformation & Loading Notebook

## Chain extraction
In order to query the complete option dataset, the desired option chains must first be collected. I collected all available chains for each stock index during each month between dec-2019 and feb-2021 using the FactSet Excel add-in(see table below). Given the limitations of the Factset add-in, running a query for all of these option chains will be too resource intensive. For this reason, I will have to make a pre-selection. First, as longer-maturity options reappear each month, I can remove duplicate chains. Second, since I will only need data on call options, I can omit put option chains from my dataset as well.

In [9]:
# defining some frequently used folders for my system
wdir = '/Users/symyion/PycharmProjects/master_thesis/'
input_dir = '/Users/symyion/PycharmProjects/master_thesis/data/input/'
output_dir = '/Users/symyion/PycharmProjects/master_thesis/data/output/'

In [11]:
import pandas as pd


pd.read_csv(input_dir+'chain_data.csv', dtype = 'str').head()

,01/12/2019,01/12/2019.1,01/12/2019.2,01/12/2019.3,01/12/2019.4,01/12/2019.5,01/12/2019.6,01/01/2020,01/01/2020.1,01/01/2020.2,...,01/01/2021.4,01/01/2021.5,01/01/2021.6,01/02/2021,01/02/2021.1,01/02/2021.2,01/02/2021.3,01/02/2021.4,01/02/2021.5,01/02/2021.6
0,181010,187653,SPX,180555,180461,180454,DIA,181010,187653,SPX,...,180461,180454,DIA,181010,187653,SPX,180555,180461,180454,DIA
1,AEX.ENX#PGKYQ,DAX.XEX#C4XS5,SPX.SPX#C02H4,UKX.FTX#C0HSJ,NIK.NKX#C18LN,PXL.ENX#C4Q6N,DIA.US#C04LV,AEX.ENX#PGKYQ,DAX.XEX#C0YQT,SPX.SPX#C07GM,...,NIK.NKX#C06CY,PXL.ENX#C17RV,DIA.US#C01DZ,AEX.ENX#C0XZR,DAX.XEX#C0C2P,SPX.SPX#C09QD,UKX.FTX#C0HSJ,NIK.NKX#C06CY,PXL.ENX#C17RV,DIA.US#C0MVV
2,AEX.ENX#CLCQ2,DAX.XEX#C71SC,SPX.SPX#C07GM,UKX.FTX#C0KYH,NIK.NKX#C1HZT,PXL.ENX#C5VJ6,DIA.US#C2LFY,AEX.ENX#CKZBH,DAX.XEX#C2TX0,SPX.SPX#C0GWD,...,NIK.NKX#C0TDL,PXL.ENX#C44P2,DIA.US#C08G9,AEX.ENX#C79XF,DAX.XEX#C2PKN,SPX.SPX#C0HPK,UKX.FTX#C0KYH,NIK.NKX#C0TDL,PXL.ENX#C44P2,DIA.US#C1MDC
3,AEX.ENX#CPPWG,DAX.XEX#C8N21,SPX.SPX#C0DZQ,UKX.FTX#C0S3C,NIK.NKX#C1JFL,PXL.ENX#CJ1L9,DIA.US#C2QPV,AEX.ENX#CNHYJ,DAX.XEX#C3BHL,SPX.SPX#C170T,...,NIK.NKX#C1RDF,PXL.ENX#CBX72,DIA.US#C1FM7,AEX.ENX#CCMQ4,DAX.XEX#C5CQ8,SPX.SPX#C12VJ,UKX.FTX#C0S3C,NIK.NKX#C1RDF,PXL.ENX#CBX72,DIA.US#C2N5X
4,AEX.ENX#CSHG6,DAX.XEX#CG41J,SPX.SPX#C0GWD,UKX.FTX#C0VB2,NIK.NKX#C1ZMP,PXL.ENX#CLQ0F,DIA.US#C3KXP,AEX.ENX#CSCNB,DAX.XEX#C3D2K,SPX.SPX#C19B9,...,NIK.NKX#C283Z,PXL.ENX#CFR97,DIA.US#C1H8X,AEX.ENX#CFNK7,DAX.XEX#C65WP,SPX.SPX#C1N04,UKX.FTX#C0VB2,NIK.NKX#C283Z,PXL.ENX#CFR97,DIA.US#C3HJJ


Within the 'tools' module, I wrote a small class (OptionChainConverter) that converts the dataframe above to a long-format file. Once all chains are in one column, it allows me to check for and remove duplicates. Moreover, based on the fact that the beginning of each option chain sequence starts with either C for call or P for put (e.g. AEX.ENX#PGKYQ or AEX.ENX#CLCQ2) the OptionChainConverter is able to filter out puts with the get_call method. 

The resulting data frame is shown below:

In [10]:
import tools


chains = tools.OptionChainConverter(csv_data_path=input_dir+'chain_data.csv')
chains.get_calls(call_path=output_dir+'call_chains.xlsx',write_excel=True).head()

Call chains saved to output folder output/call_chains.xlsx


,obs_month,index,chain_id
1,01/12/2019,181010,AEX.ENX#CLCQ2
2,01/12/2019,181010,AEX.ENX#CPPWG
3,01/12/2019,181010,AEX.ENX#CSHG6
4,01/12/2019,181010,AEX.ENX#CTZNS
5,01/12/2019,181010,AEX.ENX#CZJWL


## Description extraction

Due to the pre-selections made above, the dataset (call_chains.xlsx) is reduced from 160.000 option chains to approximately 80.0000 chains.  However, given that I will have to query roughly 300 daily observations for each chain, this still leaves me at a query size of about 24 million observations. For this reason, I will first fetch a description for each option chain using the FactSet Excel Add-in. The resulting description is shown in the table below:



In [125]:
raw_calls = pd.read_csv(input_dir+'call_descriptions.csv', dtype = 'str')
raw_calls#.head(5)

,obs_month,index,chain_id,description
0,01/12/2019,181010,AEX.ENX#CLCQ2,Netherlands AEX Call DEC19 500.00
1,01/12/2019,181010,AEX.ENX#CPPWG,Netherlands AEX Call DEC19 400.00
2,01/12/2019,181010,AEX.ENX#CSHG6,Netherlands AEX Call DEC19 200.00
3,01/12/2019,181010,AEX.ENX#CTZNS,Netherlands AEX Call DEC19 600.00
4,01/12/2019,181010,AEX.ENX#CZJWL,Netherlands AEX Call DEC19 300.00
...,...,...,...,...
83757,01/02/2021,DIA,DIA.US#CT9CT,SPDR Dow Jones Industrial Average ETF Trust Ca...
83758,01/02/2021,DIA,DIA.US#CVSCV,SPDR Dow Jones Industrial Average ETF Trust Ca...
83759,01/02/2021,DIA,DIA.US#CVYNB,SPDR Dow Jones Industrial Average ETF Trust Ca...
83760,01/02/2021,DIA,DIA.US#CWVST,SPDR Dow Jones Industrial Average ETF Trust Ca...


In [126]:
raw_calls.tail(5)

,obs_month,index,chain_id,description
83757,01/02/2021,DIA,DIA.US#CT9CT,SPDR Dow Jones Industrial Average ETF Trust Ca...
83758,01/02/2021,DIA,DIA.US#CVSCV,SPDR Dow Jones Industrial Average ETF Trust Ca...
83759,01/02/2021,DIA,DIA.US#CVYNB,SPDR Dow Jones Industrial Average ETF Trust Ca...
83760,01/02/2021,DIA,DIA.US#CWVST,SPDR Dow Jones Industrial Average ETF Trust Ca...
83761,01/02/2021,DIA,DIA.US#CY9GD,SPDR Dow Jones Industrial Average ETF Trust Ca...


Since I am only interest in 3 month+ expiration days, weekly options may be omitted. This already leaves us with half the amount of options. Moreover, since some descriptions contain flags such as '(LEAPs)' indicating far-ahead expiration dates,these flags will have to be removed from the description in order to avoid inconsistencies in the next extraction  steps.

In [150]:
def drop_weeklies (df):
    # Remove observations that have "Weekly" in their description
    no_weeklies = df.loc[~df.description.str.contains('Weekly')].copy()
    no_weeklies = no_weeklies.reset_index(drop=True)
    return no_weeklies

def drop_parenthesis_flags (df):
    # Remove anything redundant in the descriptions between brackets such as (LEAPs)
    no_flags = df.description.str.replace(r"\s*\([^()]*\)","", regex = True) 
    return no_flags

In [151]:
calls = drop_weeklies(raw_calls)
calls['description'] = drop_parenthesis_flags(calls)
calls

,obs_month,index,chain_id,description
0,01/12/2019,181010,AEX.ENX#CLCQ2,Netherlands AEX Call DEC19 500.00
1,01/12/2019,181010,AEX.ENX#CPPWG,Netherlands AEX Call DEC19 400.00
2,01/12/2019,181010,AEX.ENX#CSHG6,Netherlands AEX Call DEC19 200.00
3,01/12/2019,181010,AEX.ENX#CTZNS,Netherlands AEX Call DEC19 600.00
4,01/12/2019,181010,AEX.ENX#CZJWL,Netherlands AEX Call DEC19 300.00
...,...,...,...,...
44478,01/02/2021,DIA,DIA.US#CFSCM,SPDR Dow Jones Industrial Average ETF Trust Ca...
44479,01/02/2021,DIA,DIA.US#CL7J9,SPDR Dow Jones Industrial Average ETF Trust Ca...
44480,01/02/2021,DIA,DIA.US#CR27X,SPDR Dow Jones Industrial Average ETF Trust Ca...
44481,01/02/2021,DIA,DIA.US#CJKML,SPDR Dow Jones Industrial Average ETF Trust Ca...


Each description contains information on the option's underlying, expiration date and strike price. Some regular expression magic should be able to extract this.

In [162]:
from datetime import datetime
import calendar


def get_option_expiration(date):
    # convert Year/Month date combination to third friday of the month date.
    day = 21 - (calendar.weekday(date.year, date.month, 1) + 2) % 7
    return datetime(date.year, date.month, day)

def description_splitter(df):
    df['index_ticker'] = df['chain_id'].str[:3]
    df['strike_price'] = df['description'].str.split().str[-1]
    exp_string = df['description'].str.split().str[-2]
    exp_date_ = pd.to_datetime(exp_string, format = '%b%y')
    df['exp_date'] = exp_date_.apply(get_option_expiration)
    return df

In [164]:
calls = description_splitter(calls)
calls

,obs_month,index,chain_id,description,index_ticker,strike_price,exp_date
0,01/12/2019,181010,AEX.ENX#CLCQ2,Netherlands AEX Call DEC19 500.00,AEX,500.00,2019-12-20
1,01/12/2019,181010,AEX.ENX#CPPWG,Netherlands AEX Call DEC19 400.00,AEX,400.00,2019-12-20
2,01/12/2019,181010,AEX.ENX#CSHG6,Netherlands AEX Call DEC19 200.00,AEX,200.00,2019-12-20
3,01/12/2019,181010,AEX.ENX#CTZNS,Netherlands AEX Call DEC19 600.00,AEX,600.00,2019-12-20
4,01/12/2019,181010,AEX.ENX#CZJWL,Netherlands AEX Call DEC19 300.00,AEX,300.00,2019-12-20
...,...,...,...,...,...,...,...
44478,01/02/2021,DIA,DIA.US#CFSCM,SPDR Dow Jones Industrial Average ETF Trust Ca...,DIA,175.00,2021-04-16
44479,01/02/2021,DIA,DIA.US#CL7J9,SPDR Dow Jones Industrial Average ETF Trust Ca...,DIA,160.00,2021-04-16
44480,01/02/2021,DIA,DIA.US#CR27X,SPDR Dow Jones Industrial Average ETF Trust Ca...,DIA,185.00,2021-04-16
44481,01/02/2021,DIA,DIA.US#CJKML,SPDR Dow Jones Industrial Average ETF Trust Ca...,DIA,155.00,2021-04-16
